In [3]:
import os

In [4]:
from PIL import Image

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms

In [ ]:
class CustomImageDataset(Dataset):
    def read_data_set(self):
        
        all_img_files = []
        all_labels = []
        
        class_names = os.walk(self.data_set_path).__next__()[1]
        
        for index, class_name in enumerate(class_names):
            label = index
            img

In [8]:
import os
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms


class CustomImageDataset(Dataset):
    def read_data_set(self):

        all_img_files = []
        all_labels = []

        class_names = os.walk(self.data_set_path).__next__()[1]

        for index, class_name in enumerate(class_names):
            label = index
            img_dir = os.path.join(self.data_set_path, class_name)
            img_files = os.walk(img_dir).__next__()[2]

            for img_file in img_files:
                img_file = os.path.join(img_dir, img_file)
                img = Image.open(img_file)
                if img is not None:
                    all_img_files.append(img_file)
                    all_labels.append(label)

        return all_img_files, all_labels, len(all_img_files), len(class_names)

    def __init__(self, data_set_path, transforms=None):
        self.data_set_path = data_set_path
        self.image_files_path, self.labels, self.length, self.num_classes = self.read_data_set()
        self.transforms = transforms

    def __getitem__(self, index):
        image = Image.open(self.image_files_path[index])
        image = image.convert("RGB")

        if self.transforms is not None:
            image = self.transforms(image)

        return {'image': image, 'label': self.labels[index]}

    def __len__(self):
        return self.length


class CustomConvNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomConvNet, self).__init__()

        self.layer1 = self.conv_module(3, 16)
        self.layer2 = self.conv_module(16, 32)
        self.layer3 = self.conv_module(32, 64)
        self.layer4 = self.conv_module(64, 128)
        self.layer5 = self.conv_module(128, 256)
        self.gap = self.global_avg_pool(256, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.gap(out)
        out = out.view(-1, num_classes)

        return out

    def conv_module(self, in_num, out_num):
        return nn.Sequential(
            nn.Conv2d(in_num, out_num, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_num),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

    def global_avg_pool(self, in_num, out_num):
        return nn.Sequential(
            nn.Conv2d(in_num, out_num, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_num),
            nn.LeakyReLU(),
            nn.AdaptiveAvgPool2d((1, 1)))


hyper_param_epoch = 20
hyper_param_batch = 8
hyper_param_learning_rate = 0.001

transforms_train = transforms.Compose([transforms.Resize((128, 128)),
                                       transforms.RandomRotation(10.),
                                       transforms.ToTensor()])

transforms_test = transforms.Compose([transforms.Resize((128, 128)),
                                      transforms.ToTensor()])

train_data_set = CustomImageDataset(data_set_path="../data/git_test/train", transforms=transforms_train)
train_loader = DataLoader(train_data_set, batch_size=hyper_param_batch, shuffle=True)

test_data_set = CustomImageDataset(data_set_path="../data/git_test/test", transforms=transforms_test)
test_loader = DataLoader(test_data_set, batch_size=hyper_param_batch, shuffle=True)

if not (train_data_set.num_classes == test_data_set.num_classes):
    print("error: Numbers of class in training set and test set are not equal")
    exit()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_classes = train_data_set.num_classes
custom_model = CustomConvNet(num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(custom_model.parameters(), lr=hyper_param_learning_rate)

for e in range(hyper_param_epoch):
    for i_batch, item in enumerate(train_loader):
        images = item['image'].to(device)
        labels = item['label'].to(device)

        # Forward pass
        outputs = custom_model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i_batch + 1) % hyper_param_batch == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'
                  .format(e + 1, hyper_param_epoch, loss.item()))

# Test the model
custom_model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for item in test_loader:
        images = item['image'].to(device)
        labels = item['label'].to(device)
        outputs = custom_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += len(labels)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the {} test images: {} %'.format(total, 100 * correct / total))

Epoch [1/20], Loss: 0.5929
Epoch [2/20], Loss: 0.5780
Epoch [3/20], Loss: 0.5731
Epoch [4/20], Loss: 0.5462
Epoch [5/20], Loss: 0.5412
Epoch [6/20], Loss: 0.5328
Epoch [7/20], Loss: 0.5166
Epoch [8/20], Loss: 0.5021
Epoch [9/20], Loss: 0.5039
Epoch [10/20], Loss: 0.4870
Epoch [11/20], Loss: 0.4825
Epoch [12/20], Loss: 0.4777
Epoch [13/20], Loss: 0.4708
Epoch [14/20], Loss: 0.4517
Epoch [15/20], Loss: 0.4487
Epoch [16/20], Loss: 0.4343
Epoch [17/20], Loss: 0.4303
Epoch [18/20], Loss: 0.4636
Epoch [19/20], Loss: 0.4347
Epoch [20/20], Loss: 0.4067
Test Accuracy of the model on the 20 test images: 100.0 %


# 머신러닝, 딥러닝 실전 개발 입문
 ## 7장 이미지와 딥러닝

In [1]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.


In [1]:
from PIL import Image
import numpy as np

In [6]:
# 이미지 데이터를 Average Hash로 변환하기 --- (※1)
def average_hash(fname, size = 16):
    img = Image.open(fname) # 이미지 데이터 열기---(※2)
    img = img.convert('L') # 그레이스케일로 변환하기 --- (※3)
    img = img.resize((size, size), Image.ANTIALIAS) # 리사이즈하기 --- (※4)
    pixel_data = img.getdata() # 픽셀 데이터 가져오기 --- (※5)
    pixels = np.array(pixel_data) # Numpy 배열로 변환하기 --- (※6)
    pixels = pixels.reshape((size, size)) # 2차원 배열로 변환하기 --- (※7)
    avg = pixels.mean() # 평균 구하기 --- (※8)
    diff = 1 * (pixels > avg) # 평균보다 크면 1, 작으면 0으로 변환하기 --- (※9)
    return diff
# 이진수를 16진수 해시로 변환하기 --- (※10)
def np2hash(ahash):
    bhash = []
    for nl in ahash.tolist():
        sl = [str(i) for i in nl]
        s2 = "".join(sl)
        i = int(s2, 2) # 이진수를 정수로 변환하기
        bhash.append("%04x" % i)
    return "".join(bhash)
# Average Hash 출력하기
ahash = average_hash('../python-for-machine-learning_20190826/ch7/tower.jpg')
print(ahash)
print(np2hash(ahash))


[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0]
 [0 1 0 1 1 1 1 1 1 1 1 1 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0]
 [1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0]
 [1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1]]
000001000180018003c003c003c087e07ff05ff07ff2cffa9ffe3ffe3ffe1fff


In [4]:
# 이미지 데이터를 Average Hash로 변환하기 --- (※1)
def average_hash(fname, size = 16):
    img = Image.open(fname) # 이미지 데이터 열기---(※2)
    img = img.convert('L') # 그레이스케일로 변환하기 --- (※3)
    img = img.resize((size, size), Image.ANTIALIAS) # 리사이즈하기 --- (※4)
    pixel_data = img.getdata() # 픽셀 데이터 가져오기 --- (※5)
    pixels = np.array(pixel_data) # Numpy 배열로 변환하기 --- (※6)
    pixels = pixels.reshape((size, size)) # 2차원 배열로 변환하기 --- (※7)
   
    return pixels
# 이진 해시로 변환하기 --- (※10)
def np2hash(ahash):
    bhash = []
    for nl in ahash.tolist():
        sl = [str(i) for i in nl]
        s2 = "".join(sl)
        i = int(s2, 2) # 이진수를 정수로 변환하기
        bhash.append("%04x" % i)
    return "".join(bhash)
# Average Hash 출력하기
ahash = average_hash('../python-for-machine-learning_20190826/ch7/tower.jpg')
print(ahash)
print(np2hash(ahash))

[[  3   4   4   4   5   8  14  33  18   9   6   5   4   4   3   4]
 [  4   4   3   4   6   9  13 141  43   6   7   5   4   4   3   4]
 [  4   4   3   4   6   9  13 212 118   0  10   4   3   3   3   3]
 [  3   3   3   4   7   5  39 156 122  17   7   5   4   3   3   3]
 [  4   3   3   5   9   0  94 195 206  93   1   7   3   3   3   3]
 [  3   4   5   6   8   0  63  84 102  63   1   7   3   3   3   3]
 [  7   3   0   1   3   9  78 111 103  86  20   0   6   4   3   3]
 [ 62  53  56  51  45  61 101 111 106 101  81  19   0   3   4   4]
 [ 12  61  58  60  79  69  80 100  96  75  70  63  46  11   0   0]
 [ 13  71  41 102  84  97 138 116 121  98  81  63  54  52  41  22]
 [ 32  72  75  90  95 103 127 124 125 115 111  90  48  50  71  53]
 [ 98 100  34  55 144 117 144 122  98 122 105  88  71  37  73  54]
 [ 75  32  47  67  92 119 137 138 115 120 129 106 101  76  70  37]
 [ 39  44  77  64  76 111 162 151 109 124 143 110  76  80  88  24]
 [ 40  48  73  90 109 124 175 128 114 128 144 124  75  82  93 

ValueError: invalid literal for int() with base 2: '3444581433189654434'